In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# Fehlerminderung konfigurieren

import Tabs from '@theme/Tabs';
import TabItem from '@theme/TabItem';



> **Note:** Die Betaversion eines neuen Ausführungsmodells ist jetzt verfügbar. Das gerichtete Ausführungsmodell bietet mehr Flexibilität bei der Anpassung Ihres Fehlerminderungs-Workflows. Weitere Informationen finden Sie in der Anleitung [Gerichtetes Ausführungsmodell](/guides/directed-execution-model).
{/*
  DO NOT EDIT THIS CELL!!!
  This cell's content is generated automatically by a script. Anything you add
  here will be removed next time the notebook is run. To add new content, create
  a new cell before or after this one.
*/}

<details>
<summary><b>Paketversionen</b></summary>

Der Code auf dieser Seite wurde mit den folgenden Anforderungen entwickelt.
Wir empfehlen die Verwendung dieser oder neuerer Versionen.

```
qiskit-ibm-runtime~=0.43.1
```
</details>
Fehlerminderungstechniken ermöglichen es Benutzern, Schaltkreisfehler durch Modellierung des Geräterauschens zum Zeitpunkt der Ausführung zu mindern. Dies führt typischerweise zu einem quantenmechanischen Vorverarbeitungsaufwand im Zusammenhang mit dem Modelltraining und einem klassischen Nachverarbeitungsaufwand, um Fehler in den Rohergebnissen mithilfe des generierten Modells zu mindern.

Der Estimator-Primitive unterstützt mehrere Fehlerminderungstechniken, darunter [TREX](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/options-resilience-options-v2#measure_mitigation), [ZNE](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/options-resilience-options-v2#zne), [PEC](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/options-resilience-options-v2#pec) und [PEA](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/options-zne-options). Eine Erläuterung zu jeder Technik finden Sie unter [Fehlerminderungs- und Unterdrückungstechniken](error-mitigation-and-suppression-techniques). Bei Verwendung von Primitives können Sie einzelne Methoden ein- oder ausschalten. Details finden Sie im Abschnitt [Benutzerdefinierte Fehlereinstellungen](#advanced-error).

> **Note:** Sampler unterstützt keine Fehlerminderung, aber Sie können das [mthree](https://qiskit.github.io/qiskit-addon-mthree/)-Paket (matrixfreie Messungsminderung) verwenden, um die Fehlerminderung lokal durchzuführen.

Estimator unterstützt auch `resilience_level`. Das Resilience Level gibt an, wie viel Widerstandsfähigkeit gegen Fehler aufgebaut werden soll. Höhere Level erzeugen genauere Ergebnisse, auf Kosten längerer Verarbeitungszeiten. Resilience Levels können verwendet werden, um den Kosten-Genauigkeits-Kompromiss bei der Anwendung von Fehlerminderung auf Ihre Primitive-Abfrage zu konfigurieren. Die Fehlerminderung reduziert Fehler (Bias) in Ergebnissen durch Verarbeitung der Ausgaben aus einer Sammlung oder einem Ensemble verwandter Schaltkreise. Der Grad der Fehlerreduktion hängt von der angewendeten Methode ab. Das Resilience Level abstrahiert die detaillierte Auswahl der Fehlerminderungsmethode, um Benutzern zu ermöglichen, über den Kosten-Genauigkeits-Kompromiss nachzudenken, der für ihre Anwendung geeignet ist.

Vor diesem Hintergrund entspricht jedes Level einer oder mehreren Methoden mit zunehmendem quantenmechanischen Sampling-Aufwand, damit Sie mit verschiedenen Zeit-Genauigkeits-Kompromissen experimentieren können. Die folgende Tabelle zeigt Ihnen, welche Levels und entsprechende Methoden für jeden der Primitives verfügbar sind.

> **Info:** Die Fehlerminderung ist aufgabenspezifisch, sodass die Techniken, die Sie anwenden können, je nachdem variieren, ob Sie eine Verteilung samplen oder Erwartungswerte generieren.

<span id="resilience-table"></span>

Estimator unterstützt die folgenden Resilience Levels. Sampler unterstützt keine Resilience Levels.

| Resilience Level | Definition                                                                                            | Technik                                                                 |
|------------------|-------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------|
| 0                | Keine Minderung                                                                                       | Keine                                                                      |
| 1 [Standard]      | Minimale Minderungskosten: Fehler im Zusammenhang mit Auslese-Fehlern mindern                        | Twirled Readout Error eXtinction (TREX) Messungs-Twirling             |
| 2                | Mittlere Minderungskosten. Reduziert typischerweise Bias in Estimatoren, ist aber nicht garantiert Null-Bias zu sein. | Level 1 + Zero Noise Extrapolation (ZNE) und Gate-Twirling

> **Info:** Resilience Levels befinden sich derzeit in der Beta, sodass Sampling-Aufwand und Lösungsqualität von Schaltkreis zu Schaltkreis variieren werden. Neue Features, erweiterte Optionen und Verwaltungstools werden fortlaufend veröffentlicht. Spezifische Fehlerminderungsmethoden sind nicht garantiert auf jedem Resilience Level angewendet zu werden.

## Estimator mit Resilience Levels konfigurieren
Sie können Resilience Levels verwenden, um Fehlerminderungstechniken anzugeben, oder Sie können benutzerdefinierte Techniken einzeln festlegen, wie in [Benutzerdefinierte Fehlereinstellungen](#advanced-error) beschrieben.

<details>
<summary>Resilience Level 0</summary>

Es wird keine Fehlerminderung auf das Benutzerprogramm angewendet.

</details>

<details>
<summary>Resilience Level 1</summary>

Level 1 wendet **Auslese-Fehlerminderung** und **Messungs-Twirling** durch Anwendung einer modellfreien Technik an, die als Twirled Readout Error eXtinction (TREX) bekannt ist. Sie reduziert Messfehler, indem sie den Rauschkanal, der mit der Messung verbunden ist, diagonalisiert, indem sie Qubits durch X-Gates unmittelbar vor der Messung zufällig kippt. Ein Skalierungsterm aus dem diagonalen Rauschkanal wird durch Benchmarking zufälliger Schaltkreise gelernt, die im Null-Zustand initialisiert werden. Dies ermöglicht es dem Service, Bias aus Erwartungswerten zu entfernen, die aus Auslese-Rauschen resultieren. Dieser Ansatz wird weiter beschrieben in [Model-free readout-error mitigation for quantum expectation values](https://arxiv.org/abs/2012.09738).

</details>

<details>
<summary>Resilience Level 2</summary>

Level 2 wendet die **in Level 1 enthaltenen Fehlerminderungstechniken** an und wendet auch **Gate-Twirling** an und verwendet die **Zero Noise Extrapolation-Methode (ZNE)**. ZNE berechnet einen Erwartungswert der Observablen für verschiedene Rauschfaktoren (Amplifikationsphase) und verwendet dann die gemessenen Erwartungswerte, um den idealen Erwartungswert an der Null-Rausch-Grenze abzuleiten (Extrapolationsphase). Dieser Ansatz neigt dazu, Fehler in Erwartungswerten zu reduzieren, ist aber nicht garantiert, ein unverzerrtes Ergebnis zu erzeugen.

![This image shows a graph.  The x-axis is labeled Noise amplification factor.  The y-axis is labeled Expectation value.  An upward sloping line is labeled Mitigated value.  Points near the line are noise-amplified values.  There is a horizontal line just above the X-axis labeled Exact value.](../docs/images/guides/configure-error-mitigation/resilience-2.svg "Illustration of the ZNE method")

Der Aufwand dieser Methode skaliert mit der Anzahl der Rauschfaktoren. Die Standardeinstellungen sampeln den Erwartungswert bei drei Rauschfaktoren, was zu etwa 3x Aufwand führt, wenn dieses Resilience Level verwendet wird.

In Level 2 kippt die TREX-Methode Qubits zufällig durch X-Gates unmittelbar vor der Messung und kippt das entsprechende gemessene Bit, wenn ein X-Gate angewendet wurde. Dieser Ansatz wird weiter beschrieben in [Model-free readout-error mitigation for quantum expectation values](https://arxiv.org/abs/2012.09738).

</details>

### Beispiel
Die `EstimatorV2`-Schnittstelle ermöglicht es Benutzern, nahtlos mit einer Vielzahl von Fehlerminderungsmethoden zu arbeiten, um Fehler in Erwartungswerten von Observablen zu reduzieren. Der folgende Code verwendet Zero Noise Extrapolation und Auslese-Fehlerminderung, indem einfach `resilience_level 2` gesetzt wird.

In [1]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorV2 as Estimator

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

# Setting options during primitive initialization
estimator = Estimator(backend, options={"resilience_level": 2})

<span id="advanced-error"></span>
## Benutzerdefinierte Fehlereinstellungen
Sie können einzelne Fehlerminderungs- und Unterdrückungsmethoden ein- und ausschalten, einschließlich Dynamical Decoupling, Gate- und Messungs-Twirling, Messfehlerminderung, PEC und ZNE. Eine Erläuterung zu jeder Technik finden Sie unter [Fehlerminderungs- und Unterdrückungstechniken](error-mitigation-and-suppression-techniques).

> **Note:** - Nicht alle Optionen sind für beide Primitives verfügbar. Die Liste der verfügbaren Optionen finden Sie in der [Verfügbare-Optionen-Tabelle](runtime-options-overview#options-table).
> - Nicht alle Methoden funktionieren zusammen bei allen Arten von Schaltkreisen. Details finden Sie in der [Feature-Kompatibilitätstabelle](runtime-options-overview#options-compatibility-table).

<Tabs>
  <TabItem value="Estimator" label="Estimator">
    ```python
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorV2 as Estimator

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

estimator = Estimator(backend)
options = estimator.options
# Turn on gate twirling.
options.twirling.enable_gates = True
# Turn on measurement error mitigation.
options.resilience.measure_mitigation = True

print(f">>> gate twirling is turned on: {estimator.options.twirling.enable_gates}")
print(f">>> measurement error mitigation is turned on: {estimator.options.resilience.measure_mitigation}")
```
  </TabItem>

  <TabItem value="Sampler" label="Sampler">